### Module 1: Case Study-II : Predictive Maintenance Data set
### Attribute Information

- The dataset consists of 10,000 data points stored as rows with 14 features in columns  
- UID: unique identifier ranging from 1 to 10000  
- Product ID: consisting of a letter L, M, or H for low (50% of all products), medium (30%) and high (20%) as product quality variants and a variant-specific serial number  
- Air temperature K: generated using a random walk process later normalized to a standard deviation of 2 K around 300 K  
- Process temperature [K]: generated using a random walk process normalized to a standard deviation of 1 K, added to the air temperature plus 10 K.  
- Rotational speed [rpm]: calculated from a power of 2860 W, overlaid with a normally distributed noise  
- Torque [Nm]: torque values are normally distributed around 40 Nm with a Ïƒ = 10 Nm and no negative values.  
- Tool wear [min]: The quality variants H/M/L add 5/3/2 minutes of tool wear to the used tool in the process. and a  'machine failure' label that indicates, whether the machine has failed in this particular datapoint for any of the following failure modes are true.  
  
The machine failure consists of five independent failure modes  :
1. tool wear failure (TWF): the tool will be replaced of fail at a randomly selected tool wear time between 200 â€“ 240 mins (120 times in our dataset). At this point in time, the tool is replaced 69 times, and fails 51 times (randomly assigned).  
2. heat dissipation failure (HDF): heat dissipation causes a process failure, if the difference between air- and process temperature is below 8.6 K and the toolâ€™s rotational speed is below 1380 rpm. This is the case for 115 data points.  
3. power failure (PWF): the product of torque and rotational speed (in rad/s) equals the power required for the process. If this power is below 3500 W or above 9000 W, the process fails, which is the case 95 times in our dataset.  
4. oversdf failure (OSF): if the product of tool wear and torque exceeds 11,000 minNm for the L product variant (12,000 M, 13,000 H), the process fails due to overstrain. This is true for 98 datapoints.  
5. random failures (RNF): each process has a chance of 0,1 % to fail regardless of its process parameters. This is the case for only 5 datapoints, less than could be expected for 10,000 datapoints in our dataset.  
  
If at least one of the above failure modes is true, the process fails and the 'machine failure' label is set to 1. It is therefore not transparent to the machine learning method, which of the failure modes has caused the process to fail

### Loading Various Libraries

In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import collections
warnings.filterwarnings('ignore')

### Read Data From CSV File

In [ ]:
df=pd.read_csv('C:/Users/USER/Documents/DSB Assignments/Module 1/ai4i2020.csv',header=0)

### Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
# Basic stats and structure of data
print('Total Number of Elements: ', df.size)
print('Rows x Columns: ',df.shape)
print("Columns: ",df.columns)
display(df.info())

In [ ]:
df[[df.columns[2]]].value_counts()

#### EDA: Column Types
**High Cardinatilty Columns**: UDI, Product ID  
  
**Categorical Columns**: Type  
  
**Continuous Columns**: Air Temp, Process Temp, Rotational Speed, Torque, Tool Wear  
  
**Binary Encoded**: Machine Failure, TWF, HDF, PWF, OSF, RNF  

In [ ]:
plt.figure(figsize = (25, 15))
plotnumber = 1

for column in df.columns[3:]:
    if plotnumber <= len(df.columns):
        ax = plt.subplot(4,3, plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column, fontsize = 15)
        
    plotnumber += 1
    
plt.tight_layout()
plt.show()

### Univariate Analysis For All Features

#### Univariate Analysis: Scatter Plot

In [ ]:
plt.figure(figsize = (25, 15))
plotnumber = 1

for column in df.columns[3:]:
    if plotnumber <= len(df.columns):
        ax = plt.subplot(4,3, plotnumber)
        plt.scatter(df.index,df[column])
        plt.xlabel(column, fontsize = 15)
        
    plotnumber += 1
    
plt.tight_layout()
plt.show()

#### Univariate Analysis: Histogram

In [ ]:
df.hist(figsize=(20,20))
plt.show()

#### Univariate Analysis: Univariate Statistics

In [ ]:
for column in df.columns[4:]:
    print('Feature:', column)
    print('Mean: ',df[column].mean())
    print('Median: ',df[column].median())
    print('Mode: ',df[column].mode())
    print('Std: ',df[column].mode())
    print('Min: ',df[column].min())
    print('Max: ',df[column].max())
    print('Q1:',df[column].quantile(0.25))
    print('Q3:',df[column].quantile(0.75))
    print('IQR:',df[column].quantile(0.75)-df[column].quantile(0.25))
    print('Value count:', df[column].value_counts())
    print('-------------------------------------------')

#### Univariate Analysis: Various Metrics

In [ ]:
df.describe()

### Bivariate Analysis

In [ ]:
# print(train.corr())
sns.set(font_scale=1.0)
sns.heatmap(df.corr(),cbar=True,annot=False,cmap='YlGnBu')

### Missing Value Treatment
#### Checking for Missing Values

We see that the data is complete without any missing values. This can be visually observed as well from the heatamp. So no missing value treatment i.e columns need to be dropped or values imputed.

In [ ]:
# Shows number of missing values in each column
df.isnull().sum()
# Shows Only Columsn with Missing Values
df.columns[df.isnull().any()]

In [ ]:
sns.heatmap(df.isnull(),cbar=False,cmap='viridis')

### Outlier Treatment

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
notFailed =  df['Machine failure'].value_counts()[0]
failed = df['Machine failure'].value_counts()[1]
y = np.array([df['Machine failure'].value_counts()[0],df['Machine failure'].value_counts()[1]])
mylabels = ["Not-Failure: "+str(df['Machine failure'].value_counts()[0]), "Failure: "+ str(df['Machine failure'].value_counts()[1])]
plt.pie(y,labels = mylabels)
plt.show() 
print('% of Failures', (failed/notFailed)*100 )

We'll remove any observations that is not in the range  
$Q1 - 1.5*IQR < x < Q3 + 1.5*IQR$

In [ ]:
columnsToRemoveOutliers = ['Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]']

for i,column in enumerate(columnsToRemoveOutliers):
    # column = train.columns[0]
    plt.figure(i)
    sns.boxplot(x=df[column])
    lowerRange = df[column].quantile(0.25)-1.5*(df[column].quantile(0.75)-df[column].quantile(0.25))
    upperRange = df[column].quantile(0.75)+1.5*(df[column].quantile(0.75)-df[column].quantile(0.25))
    # lowerRange = train[column].mean() - 30*(train[column].std())
    # upperRange = train[column].mean() + 30*(train[column].std())
    # plt.scatter(train.index,train[column])
    # plt.xlabel(column)
    # plt.show()
    print('-------')
    print('Columns Name:',column)
    print("Non-Outlier Range:",lowerRange,',',upperRange)
    print("Initial Size:",df.size)
    df = df[(df[column]<upperRange) & (df[column]>lowerRange)]
    print("Final Size:",df.size)
    print('-------')
    # plt.scatter(train.index,train[column])
    # plt.xlabel(column)
    # plt.show()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
y = np.array([df['Machine failure'].value_counts()[0],df['Machine failure'].value_counts()[1]])
mylabels = ["Not-Failure: "+str(df['Machine failure'].value_counts()[0]), "Failure: "+ str(df['Machine failure'].value_counts()[1])]
plt.pie(y,labels = mylabels)
plt.show() 
print('% of Failures', (failed/notFailed)*100 )

### Feature Engineering
#### Dropping Columns with High Cardinality

In [ ]:
print(df.columns)

In [ ]:
df = df.drop(['UDI', 'Product ID'], axis=1)

#### Create Columns (One-Hot Encoding)

In [ ]:
df['Type_L'] = (np.where(df['Type']=='L',1,0))
df['Type_M'] = (np.where(df['Type']=='M',1,0))
df['Type_H'] = (np.where(df['Type']=='H',1,0))

#### Selecting Columns For Logistic Regression

Removing Columns for machine failure as well as other failure columns  
Removing categorical column for Type

In [ ]:
regressionColumns = [i for i in df.columns if i not in ['Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF','Type']]


print(regressionColumns)
df[regressionColumns].head()

### Logistic Regression
#### Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df,test_size = 0.3)

#### Iterative Logistic Regression Modeling

In [ ]:
import statsmodels.api as sm
logit_model = sm.Logit(train['Machine failure'],train[regressionColumns])
result = logit_model.fit()
print(result.summary2())

In [ ]:
np.exp(result.params)

#### Adding Intercept to Model and Rerunning Logistic regression

In [ ]:
train['intercept'] =1.0
test['intercept'] =1.0

In [ ]:
regressionColumns.append('intercept')
print(regressionColumns)


In [ ]:
import statsmodels.api as sm
logit_model = sm.Logit(train['Machine failure'],train[regressionColumns])
result = logit_model.fit()
print(result.summary2())

We see intercept, Type_L, Type_M and Type_H  not giving useful results so we'll exclude those columns

In [ ]:
regressionColumns.remove('Type_L')
regressionColumns.remove('Type_M')
regressionColumns.remove('Type_H')

In [ ]:
import statsmodels.api as sm
logit_model = sm.Logit(train['Machine failure'],train[regressionColumns])
result = logit_model.fit()
print(result.summary2())

In [ ]:
regressionColumns.remove('intercept')

In [ ]:
import statsmodels.api as sm
logit_model = sm.Logit(train['Machine failure'],train[regressionColumns])
result = logit_model.fit()
print(result.summary2())

In [ ]:
predicted = result.predict(test[regressionColumns])
predictions_bin = (predicted>0.5).astype(int)
print((predictions_bin[:5],predicted[:5]))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(confusion_matrix(test['Machine failure'],predictions_bin))
print(classification_report(test['Machine failure'],predictions_bin))

### Plot ROC Curve

In [ ]:
from sklearn.linear_model import LogisticRegression
# create an instance and fit the model
logreg = LogisticRegression()
logreg.fit(train[regressionColumns], train['Machine failure'])


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(test['Machine failure'],logreg.predict(test[regressionColumns]))
fpr, tpr, thresholds = roc_curve(test['Machine failure'], logreg.predict_proba(test[regressionColumns])[:,1])
plt.figure()
plt.plot(fpr,tpr,label = 'Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0,1],[0,1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0,1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.title('Receiver Operating Characteristics')
plt.legend(loc='lower right')
plt.savefig('Log_ROC')
plt.show()


### Improving Model using WOE and IV Approach

In [ ]:

import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string


max_bin = 20
force_bin = 3

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [ ]:
final_iv, IV = data_vars(train[regressionColumns], train['Machine failure'])


In [ ]:
final_iv.head()

In [ ]:
IV.sort_values('IV')

#### Replacing Original Variables with WOE Variables

In [ ]:
transform_vars_list = train.columns.difference(['Machine','Type_H','Type_L', 'Type_M',
'intercept','Type', 'OSF', 'PWF','HDF','RNF','TWF'])
transform_prefix = 'new_'

In [ ]:
print(transform_vars_list)

In [ ]:
print(final_iv)

In [ ]:
for var in transform_vars_list:
    small_df = final_iv[final_iv['VAR_NAME'] == var]
    transform_dict = dict(zip(small_df.MAX_VALUE,small_df.WOE))
    replace_cmd = ''
    replace_cmd1 = ''
    for i in sorted(transform_dict.items()):
        replace_cmd = replace_cmd + str(i[1]) + str(' if x <= ') + str(i[0]) + ' else '
        replace_cmd1 =  replace_cmd1 + str(i[1]) + str(' if x== "') + str(i[0]) + '" else '
    replace_cmd = replace_cmd + '0'
    replace_cmd1 = replace_cmd1 + '0'
    if replace_cmd !='0':
        try:
            train[transform_prefix + var] = train[var].apply(lambda x: eval(replace_cmd))
        except:
            train[transform_prefix + var] = train[var].apply(lambda x: eval(replace_cmd1))

In [ ]:
print(train.columns)

In [ ]:
newRegressionColumns = [i for i in train.columns if 'new' in i]
print(newRegressionColumns)

In [ ]:
logit_model = sm.Logit(train['Machine failure'], train[newRegressionColumns])
result = logit_model.fit()
print(result.summary2())

In [ ]:
np.exp(result.params)